In [33]:
import os
import subprocess
from astropy.table import Table
import numpy as np
os.environ['DECALS_PATH'] = '/global/project/projectdirs/cosmo/data/legacysurvey/dr6/'
minitestdir = os.getcwd()
minitestdata = minitestdir.replace('src', 'data')
targetfile = os.path.join(minitestdata, 'dr6_targets.fits')
print(minitestdir)

/global/cscratch1/sd/forero/decals_fiberaassign_hack/src


In [26]:
cmd = "select_targets {source} {destination}"
cmd = cmd.format(source=os.environ['DECALS_PATH'], destination=targetfile)
print(cmd)

select_targets /global/project/projectdirs/cosmo/data/legacysurvey/dr6/ /global/cscratch1/sd/forero/decals_fiberaassign_hack/data/dr6_targets.fits


In [27]:
subprocess.call(cmd.split())

0

In [29]:
targetdata = Table.read(targetfile)

In [31]:
targetdata[:10]

RELEASE,BRICKID,BRICKNAME,BRICK_OBJID,TYPE,RA,DEC,RA_IVAR,DEC_IVAR,DCHISQ [5],FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,FLUX_IVAR_W3,FLUX_IVAR_W4,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,MW_TRANSMISSION_W3,MW_TRANSMISSION_W4,NOBS_G,NOBS_R,NOBS_Z,FRACFLUX_G,FRACFLUX_R,FRACFLUX_Z,ALLMASK_G,ALLMASK_R,ALLMASK_Z,PSFDEPTH_G,PSFDEPTH_R,PSFDEPTH_Z,GALDEPTH_G,GALDEPTH_R,GALDEPTH_Z,FRACDEV,FRACDEV_IVAR,SHAPEDEV_R,SHAPEDEV_R_IVAR,SHAPEDEV_E1,SHAPEDEV_E1_IVAR,SHAPEDEV_E2,SHAPEDEV_E2_IVAR,SHAPEEXP_R,SHAPEEXP_R_IVAR,SHAPEEXP_E1,SHAPEEXP_E1_IVAR,SHAPEEXP_E2,SHAPEEXP_E2_IVAR,SUBPRIORITY,PHOTSYS,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,HPXPIXEL
int32,int32,str8,int32,str4,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str1,int64,int64,int64,int64,int64
6000,492431,1475p292,5,DEV,147.571134354,29.2325914458,8.30913e+10,1.3693e+11,10369.8 .. 12692.1,7.97411,23.2207,0.0,42.7246,30.7484,8.49282,470.423,36.9622,21.4089,0.0,2.02367,0.388105,0.000992943,2.36815e-05,0.945964,0.963272,0.979287,0.996825,0.998049,0.999584,0.999843,1,1,0,0.0,0.0,0.0,0,0,0,245.78,144.602,0.0,155.891,93.5388,0.0,1.0,1.0,1.48173,1820.84,-0.0775595,4597.68,0.0790266,4814.47,0.0,0.0,0.0,0.0,0.0,0.0,0.436934,N,26390344471937029,1152921504606846976,131074,0,5288
6000,492431,1475p292,7,EXP,147.47009862,29.2339058275,1.21617e+11,4.44768e+11,16278.3 .. 24308.1,2451.18,4161.93,0.0,1272.62,684.487,68.9148,413.124,0.00316357,0.000614313,0.0,2.69431,0.513504,0.00106597,2.33093e-05,0.938881,0.958407,0.976517,0.996396,0.997785,0.999527,0.999821,1,1,0,0.0,0.0,0.0,2,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.85426,31973.0,-0.203119,32821.3,0.00118753,26353.6,0.419708,N,26390344471937031,1152921504606846976,131074,0,5288
6000,492431,1475p292,8,EXP,147.444876679,29.2348525267,1.24563e+11,3.29213e+11,15406.5 .. 21239.2,1869.98,2717.11,0.0,834.151,455.018,48.9125,-378.272,0.00471102,0.00120593,0.0,2.73013,0.509445,0.00115963,2.52313e-05,0.937909,0.957738,0.976136,0.996337,0.997749,0.999519,0.999819,1,1,0,0.000695014,0.000829101,0.0,2,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.750187,29231.1,-0.146507,16446.7,0.138534,20067.2,0.341805,N,26390344471937032,1152921504606846976,131074,0,5288
6000,492431,1475p292,10,EXP,147.444425657,29.2347231365,1.8803e+08,1.1916e+09,331.369 .. 52.1284,38.5262,66.2706,0.0,10.3198,-15.7462,196.401,1684.21,0.691063,0.394265,0.0,0.282095,0.0637983,0.000228606,8.38947e-06,0.937921,0.957747,0.976141,0.996338,0.997749,0.99952,0.999819,1,1,0,150.966,126.579,0.0,2,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0147,14.0922,-0.382303,6248.46,0.17441,3618.7,0.985431,N,26390344471937034,1152921504606846976,131074,0,5288
6000,492431,1475p292,15,REX,147.508608773,29.2324340078,8.00277e+10,1.19672e+11,10565.4 .. 12623.5,7.42092,17.092,0.0,45.3917,41.4823,201.799,-35.1039,59.8826,37.0304,0.0,2.62357,0.495141,0.00120417,2.66632e-05,0.941166,0.959978,0.977412,0.996535,0.99787,0.999545,0.999828,1,1,0,0.0,0.0,0.0,0,0,0,245.78,144.602,0.0,155.891,93.5388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.852315,9485.35,0.0,0.0,0.0,0.0,0.460817,N,26390344471937039,1152921504606846976,131074,0,5288
6000,492431,1475p292,47,COMP,147.48878798,29.237059988,5.75113e+10,6.4055e+10,8783.04 .. 13081.9,109.432,154.823,0.0,480.923,254.51,137.215,178.051,0.563443,0.271883,0.0,2.03525,0.393372,0.000967826,2.24674e-05,0.939633,0.958924,0.976812,0.996442,0.997813,0.999533,0.999824,1,1,0,5.52727e-05,6.70682e-05,0.0,2,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.554079,9

In [32]:
np.count_nonzero(targetdata['MWS_TARGET']!=0)

NameError: name 'np' is not defined